In [9]:
import torch
import torch.nn as nn

from typing import Any


class SkipGram(nn.Module):
    def __init__(self, vocab_size, embedding_dim):
        super(SkipGram, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.output = nn.Linear(embedding_dim, vocab_size, bias=False)

    def forward(self, center_words):
        embeds = self.embedding(center_words)
        out = self.output(embeds)
        return out


def stat_raw_text(raw_text: list[str]):
    vocab = set(raw_text)
    vocab_size = len(vocab)
    word2idx = {word: i for i, word in enumerate(vocab)}
    idx2word = {i: word for i, word in enumerate(vocab)}
    return vocab, vocab_size, word2idx, idx2word


def make_train_data(raw_text: list[str], window_size: int):
    data = []
    test_data = []
    for i in range(window_size, len(raw_text) - window_size):
        center_word = raw_text[i]
        indices = list(range(i - window_size, i)) + \
            list(range(i + 1, i + window_size + 1))

        test_data.append((center_word, [raw_text[j] for j in indices]))
        for j in indices:
            data.append((center_word, raw_text[j]))
    return data, test_data


def word_to_idx_tensor(word: str, word2idx: dict[str, int]) -> torch.Tensor:
    return torch.tensor([word2idx[word]], dtype=torch.long)


raw_text: list[str] = """We are about to study the idea of a computational process.
    Computational processes are abstract beings that inhabit computers.
    As they evolve, processes manipulate other abstract things called data.
    The evolution of a process is directed by a pattern of rules
    called a program. People create programs to direct processes. In effect,
    we conjure the spirits of the computer with our spells.""".split()  # type: ignore

vocab, vocab_size, word2idx, idx2word = stat_raw_text(raw_text)
data,  test_data = make_train_data(raw_text, 2)

input_indices = torch.tensor([word2idx[center]
                             for center, _ in data], dtype=torch.long)
output_indices = torch.tensor([word2idx[out]
                              for _, out in data], dtype=torch.long)

epochs = 500
embedding_dim = 300
learning_rate = 0.001
model = SkipGram(vocab_size, embedding_dim)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(input_indices)
    loss = criterion(outputs, output_indices)
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch}, Loss: {loss.item()}")

Epoch: 0, Loss: 4.00478982925415
Epoch: 1, Loss: 3.8974781036376953
Epoch: 2, Loss: 3.792513132095337
Epoch: 3, Loss: 3.6899876594543457
Epoch: 4, Loss: 3.589993715286255
Epoch: 5, Loss: 3.4926207065582275
Epoch: 6, Loss: 3.3979551792144775
Epoch: 7, Loss: 3.3060879707336426
Epoch: 8, Loss: 3.217111349105835
Epoch: 9, Loss: 3.13112211227417
Epoch: 10, Loss: 3.0482194423675537
Epoch: 11, Loss: 2.9685003757476807
Epoch: 12, Loss: 2.8920562267303467
Epoch: 13, Loss: 2.8189687728881836
Epoch: 14, Loss: 2.749305009841919
Epoch: 15, Loss: 2.6831107139587402
Epoch: 16, Loss: 2.6204099655151367
Epoch: 17, Loss: 2.5612003803253174
Epoch: 18, Loss: 2.505448818206787
Epoch: 19, Loss: 2.4530937671661377
Epoch: 20, Loss: 2.4040443897247314
Epoch: 21, Loss: 2.3581838607788086
Epoch: 22, Loss: 2.3153703212738037
Epoch: 23, Loss: 2.2754456996917725
Epoch: 24, Loss: 2.2382397651672363
Epoch: 25, Loss: 2.203578233718872
Epoch: 26, Loss: 2.171287775039673
Epoch: 27, Loss: 2.1412012577056885
Epoch: 28, Lo

In [8]:
def same_count(list1, list2):
    count = 0
    for word in list1:
        if word in list2:
            count += 1
    return count


def sorted_list(a):
    return sorted(a, key=lambda x: x[0])


for center_word, contexts in test_data:
    center_word = word_to_idx_tensor(center_word, word2idx)
    output = model(center_word)
    indices = torch.topk(output[0], 4).indices.tolist()
    predicted_words = [idx2word[idx] for idx in indices]
    print(
        f"Context: {sorted(contexts)}, Predicted: {sorted(predicted_words)}, accuracy: {same_count(contexts, predicted_words)}")

Context: ['We', 'are', 'study', 'to'], Predicted: ['We', 'are', 'study', 'to'], accuracy: 4
Context: ['about', 'are', 'study', 'the'], Predicted: ['about', 'direct', 'programs', 'study'], accuracy: 2
Context: ['about', 'idea', 'the', 'to'], Predicted: ['about', 'idea', 'the', 'to'], accuracy: 4
Context: ['idea', 'of', 'study', 'to'], Predicted: ['computer', 'conjure', 'of', 'spirits'], accuracy: 1
Context: ['a', 'of', 'study', 'the'], Predicted: ['a', 'of', 'study', 'the'], accuracy: 4
Context: ['a', 'computational', 'idea', 'the'], Predicted: ['a', 'called', 'computer', 'the'], accuracy: 2
Context: ['computational', 'idea', 'of', 'process.'], Predicted: ['directed', 'is', 'of', 'pattern'], accuracy: 1
Context: ['Computational', 'a', 'of', 'process.'], Predicted: ['Computational', 'a', 'of', 'process.'], accuracy: 4
Context: ['Computational', 'a', 'computational', 'processes'], Predicted: ['Computational', 'a', 'computational', 'processes'], accuracy: 4
Context: ['are', 'computational'